# Data Transform

In this notebook, we will ask you a series of questions to evaluate your findings from your EDA. Based on your response & justification, we will ask you to also apply a subsequent data transformation. 

If you state that you will not apply any data transformations for this step, you must **justify** as to why your dataset/machine-learning does not require the mentioned data preprocessing step.

The bonus step is completely optional, but if you provide a sufficient feature engineering step in this project we will add `1000` points to your Kahoot leaderboard score.

You will write out this transformed dataframe as a `.csv` file to your `data/` folder.

**Note**: Again, note that this dataset is quite large. If you find that some data operations take too long to complete on your machine, simply use the `sample()` method to transform a subset of your data.

In [149]:
# SMOTE Oversampling and Scaling
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

# Import modules for data manipulation
from sklearn.model_selection import train_test_split

# Pandas for data manipulation
import pandas as pd
# Numpy for numerica operations
import numpy as np

## Q1

Does your model contain any missing values or "non-predictive" columns? If so, which adjustments should you take to ensure that your model has good predictive capabilities? Apply your data transformations (if any) in the code-block below.

Answer here

In [150]:
# There are no missing values, but there are non-proedictive columns: nameOrig, nameDest, and isFlaggedFraud. 
# dropping these columns for simplicity and model training/performance
transacts_df = pd.read_csv('../data/bank_transactions.csv')
transacts_df.drop(columns=['nameOrig', 'nameDest', 'isFlaggedFraud'], inplace=True)
transacts_df.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,PAYMENT,983.09,36730.24,35747.15,0.00,0.00,0
1,PAYMENT,55215.25,99414.00,44198.75,0.00,0.00,0
2,CASH_IN,220986.01,7773074.97,7994060.98,924031.48,703045.48,0
3,TRANSFER,2357394.75,0.00,0.00,4202580.45,6559975.19,0
4,CASH_OUT,67990.14,0.00,0.00,625317.04,693307.19,0


In [151]:
# rename coloumn for consistency
transacts_df =  transacts_df.rename(columns={'oldbalanceOrg' : 'oldbalanceOrig'})

In [152]:
# create sample of original dataset
# Sampling 20% from each class while preserving ratio
df_sample = transacts_df.groupby('isFraud', group_keys=False).apply(
    lambda x: x.sample(frac=0.2, random_state=42)
)
df_sample.head()

C:\Users\oneps\AppData\Local\Temp\ipykernel_11628\2928600020.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sample = transacts_df.groupby('isFraud', group_keys=False).apply(


,type,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
224997,CASH_IN,40490.46,100928.00,141418.46,242868.95,272554.26,0
851624,CASH_OUT,350188.68,5056.00,0.00,4707.66,354896.34,0
733890,PAYMENT,2224.85,2578.65,353.80,0.00,0.00,0
656179,CASH_IN,170020.34,2096782.65,2266802.99,2668162.89,2861708.82,0
347436,CASH_IN,261231.80,2215631.50,2476863.31,493557.37,232325.57,0


In [153]:
df_baseline = df_sample.drop(columns='type')
df_baseline.head()


,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
224997,40490.46,100928.00,141418.46,242868.95,272554.26,0
851624,350188.68,5056.00,0.00,4707.66,354896.34,0
733890,2224.85,2578.65,353.80,0.00,0.00,0
656179,170020.34,2096782.65,2266802.99,2668162.89,2861708.82,0
347436,261231.80,2215631.50,2476863.31,493557.37,232325.57,0


## Q2

Do certain transaction types consistently differ in amount or fraud likelihood? If so, how might you transform the type column to make this pattern usable by a machine learning model? Apply your data transformations (if any) in the code-block below.

According to EDA findings, the CASH_OUT and TRANSFER types are most commonly tied to fraudulent transactions
 - Below I use of one-hot enconding to transform the 'type' column into multiple binary columns for subsequent modeling

In [154]:
# One-hot enconding to transform the 'type' column into multiple binary columns for subsequent modeling
df_sample = pd.get_dummies(df_sample, columns=['type'], prefix='', prefix_sep='', drop_first=True)
df_sample.head()

,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,CASH_OUT,DEBIT,PAYMENT,TRANSFER
224997,40490.46,100928.00,141418.46,242868.95,272554.26,0,False,False,False,False
851624,350188.68,5056.00,0.00,4707.66,354896.34,0,True,False,False,False
733890,2224.85,2578.65,353.80,0.00,0.00,0,False,False,True,False
656179,170020.34,2096782.65,2266802.99,2668162.89,2861708.82,0,False,False,False,False
347436,261231.80,2215631.50,2476863.31,493557.37,232325.57,0,False,False,False,False


In [ ]:
df_baseline_hot_encode = df_sample.copy()
df_baseline_hot_encode

,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,CASH_OUT,DEBIT,PAYMENT,TRANSFER
224997,40490.46,100928.00,141418.46,242868.95,272554.26,0,False,False,False,False
851624,350188.68,5056.00,0.00,4707.66,354896.34,0,True,False,False,False
733890,2224.85,2578.65,353.80,0.00,0.00,0,False,False,True,False
656179,170020.34,2096782.65,2266802.99,2668162.89,2861708.82,0,False,False,False,False
347436,261231.80,2215631.50,2476863.31,493557.37,232325.57,0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
154321,103368.58,103368.58,0.00,0.00,103368.58,1,True,False,False,False
570478,1735647.47,1735647.47,0.00,0.00,0.00,1,False,False,False,True
615607,10000000.00,10000000.00,0.00,211048.11,10211048.11,1,True,False,False,False
220387,1171751.17,1171751.17,0.00,0.00,0.00,1,False,False,False,True


## Q3

After exploring your data, you may have noticed that fraudulent transactions are rare compared to non-fraudulent ones. What challenges might this pose when training a machine learning model? What strategies could you use to ensure your model learns meaningful patterns from the minority class? Apply your data transformations (if any) in the code-block below.

### Challenges:

Fraudulent transactions are rare, ~0.13%, which creates a class imbalance problem. This can cause models to ignore the minority class--fraud--and achieve misleadingly high accuracy.
- Model may predict all transactions as non-fraud (99% accuracy but 0% fraud detection)
- Harder to learn minority patterns

### Possible Strategies:
- Oversample fraud cases (SMOTE) when modeling features
- Apply class weights in models
- Focus on precision, recall, F1, and AUC
- Feature engineering to amplify fraud signals
  - Apply Log-Tranformation?
  - data encoding (one-hot or dummy encoding)

In [126]:
# Pre check of using SMOTE on training data
# This will be used in model_train.ipynb
X = df_sample.drop(columns=['isFraud'])
y = df_sample['isFraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

print(" Use of SMOTE to balance Fraud and no-Fraud classes.")
y_resampled.value_counts()

 Use of SMOTE to balance Fraud and no-Fraud classes.


isFraud
0    159793
1    159793
Name: count, dtype: int64

In [128]:
df_sample

,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,CASH_OUT,DEBIT,PAYMENT,TRANSFER
224997,40490.46,100928.00,141418.46,242868.95,272554.26,0,False,False,False,False
851624,350188.68,5056.00,0.00,4707.66,354896.34,0,True,False,False,False
733890,2224.85,2578.65,353.80,0.00,0.00,0,False,False,True,False
656179,170020.34,2096782.65,2266802.99,2668162.89,2861708.82,0,False,False,False,False
347436,261231.80,2215631.50,2476863.31,493557.37,232325.57,0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
154321,103368.58,103368.58,0.00,0.00,103368.58,1,True,False,False,False
570478,1735647.47,1735647.47,0.00,0.00,0.00,1,False,False,False,True
615607,10000000.00,10000000.00,0.00,211048.11,10211048.11,1,True,False,False,False
220387,1171751.17,1171751.17,0.00,0.00,0.00,1,False,False,False,True


In [130]:
X_resampled.head()

,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,540669.74,15577.0,0.00,73810.24,788499.47,False,False,False,True
1,13165.94,672983.0,659817.06,0.00,0.00,False,False,True,False
2,280328.39,0.0,0.00,12245720.02,12526048.40,True,False,False,False
3,952.15,20261.0,19308.85,0.00,0.00,False,False,True,False
4,269646.26,20881.0,0.00,0.00,269646.26,False,False,False,True


In [91]:
# Log transforming predictor variables--those that are highly skewed, which are all in this case--for subsequent model training. 
# Columns with skew to log-transform
skewed_cols = ['amount', 'oldbalanceOrig', 'newbalanceOrig', 
               'oldbalanceDest', 'newbalanceDest']

# Apply log1p to avoid log(0)
for col in skewed_cols:
    df_sample[f'{col}_log'] = np.log1p(df_sample[col])

df_sample[['amount', 'amount_log']].head()


,amount,amount_log
224997,40490.46,10.608846
851624,350188.68,12.766230
733890,2224.85,7.707894
656179,170020.34,12.043679
347436,261231.80,12.473167


In [105]:
# Show first few rows of original & log-transformed values side by side
log_transform_save = df_sample[[col for col in skewed_cols] + [f"{col}_log" for col in skewed_cols]].head()
log_transform_save

,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,amount_log,oldbalanceOrig_log,newbalanceOrig_log,oldbalanceDest_log,newbalanceDest_log
224997,40490.46,100928.00,141418.46,242868.95,272554.26,10.608846,11.522173,11.859486,12.400281,12.515597
851624,350188.68,5056.00,0.00,4707.66,354896.34,12.766230,8.528529,0.000000,8.457159,12.779584
733890,2224.85,2578.65,353.80,0.00,0.00,7.707894,7.855409,5.871554,0.000000,0.000000
656179,170020.34,2096782.65,2266802.99,2668162.89,2861708.82,12.043679,14.555915,14.633881,14.796901,14.866930
347436,261231.80,2215631.50,2476863.31,493557.37,232325.57,12.473167,14.611048,14.722504,13.109396,12.355899


## Bonus (optional)

Are there interaction effects between variables (e.g., fraud and high amount and transaction type) that aren't captured directly in the dataset? Would it be helpful to manually engineer any new features that reflect these interactions? Apply your data transformations (if any) in the code-block below.

The original dataset doesn’t directly capture account balance changes relative to transaction amounts. To address this, I created the following features:

- **balance_delta_orig**: Captures the difference between old balance, new balance, and transaction amount.
- **balance_delta_dest**: Captures the change in destination balance minus amount.
- **amount_ratio_orig_balance**: Captures ratio of transaction amount to source account’s starting balance.
- **zero_balance_flag**: Captures binary flag for transactions that zero out the source account.

These engineered features help highlight patterns where account balances behave abnormally--behavior often associated with fraudulent activity.


In [93]:
# write out newly transformed dataset to your folder
# Shows unexpected leftover funds
df_sample['balance_delta_orig'] = (
    df_sample['oldbalanceOrig'] - df_sample['newbalanceOrig'] - df_sample['amount']
)

# Flags unusual destination account changes
df_sample['balance_delta_dest'] = (
    df_sample['newbalanceDest'] - df_sample['oldbalanceDest'] - df_sample['amount']
)

# Detects large withdrawals relative to balance
df_sample['amount_ratio_orig_balance'] = (
    df_sample['amount'] / (df_sample['oldbalanceOrig'] + 1)  # +1 avoids division by zero
)

# Flags full account drain (vectorized if-else)
df_sample['zero_balance_flag'] = (df_sample['newbalanceOrig'] == 0).astype(int)

df_sample[['balance_delta_orig', 'balance_delta_dest', 
           'amount_ratio_orig_balance', 'zero_balance_flag']].head()


,balance_delta_orig,balance_delta_dest,amount_ratio_orig_balance,zero_balance_flag
224997,-80980.92,-1.080515e+04,0.401178,0
851624,-345132.68,5.820766e-11,69.248305,1
733890,0.00,-2.224850e+03,0.862462,0
656179,-340040.68,2.352559e+04,0.081086,0
347436,-522463.61,-5.224636e+05,0.117904,0


In [94]:
df_baseline.head()

,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
224997,40490.46,100928.00,141418.46,242868.95,272554.26,0
851624,350188.68,5056.00,0.00,4707.66,354896.34,0
733890,2224.85,2578.65,353.80,0.00,0.00,0
656179,170020.34,2096782.65,2266802.99,2668162.89,2861708.82,0
347436,261231.80,2215631.50,2476863.31,493557.37,232325.57,0


In [95]:
# Another approach for selecting baseline dataset vs. transformed dataset
# Features dropped to get baseline V1
engineered_cols = [
    'balance_delta_orig', 
    'balance_delta_dest', 
    'amount_ratio_orig_balance', 
    'zero_balance_flag',
    'CASH_OUT',
    'DEBIT',
    'PAYMENT',
    'TRANSFER'
]

# V1: Original features only
df_log_transform = df_sample.drop(columns=engineered_cols)

# V2: Original + engineered (change in transactions) features
df_enhanced = df_sample.copy()


In [96]:
# 
df_log_transform

,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,amount_log,oldbalanceOrig_log,newbalanceOrig_log,oldbalanceDest_log,newbalanceDest_log
224997,40490.46,100928.00,141418.46,242868.95,272554.26,0,10.608846,11.522173,11.859486,12.400281,12.515597
851624,350188.68,5056.00,0.00,4707.66,354896.34,0,12.766230,8.528529,0.000000,8.457159,12.779584
733890,2224.85,2578.65,353.80,0.00,0.00,0,7.707894,7.855409,5.871554,0.000000,0.000000
656179,170020.34,2096782.65,2266802.99,2668162.89,2861708.82,0,12.043679,14.555915,14.633881,14.796901,14.866930
347436,261231.80,2215631.50,2476863.31,493557.37,232325.57,0,12.473167,14.611048,14.722504,13.109396,12.355899
...,...,...,...,...,...,...,...,...,...,...,...
154321,103368.58,103368.58,0.00,0.00,103368.58,1,11.546066,11.546066,0.000000,0.000000,11.546066
570478,1735647.47,1735647.47,0.00,0.00,0.00,1,14.366892,14.366892,0.000000,0.000000,0.000000
615607,10000000.00,10000000.00,0.00,211048.11,10211048.11,1,16.118096,16.118096,0.000000,12.259846,16.138981
220387,1171751.17,1171751.17,0.00,0.00,0.00,1,13.974011,13.974011,0.000000,0.000000,0.000000


In [97]:
df_enhanced


,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,CASH_OUT,DEBIT,PAYMENT,TRANSFER,amount_log,oldbalanceOrig_log,newbalanceOrig_log,oldbalanceDest_log,newbalanceDest_log,balance_delta_orig,balance_delta_dest,amount_ratio_orig_balance,zero_balance_flag
224997,40490.46,100928.00,141418.46,242868.95,272554.26,0,False,False,False,False,10.608846,11.522173,11.859486,12.400281,12.515597,-80980.92,-1.080515e+04,0.401178,0
851624,350188.68,5056.00,0.00,4707.66,354896.34,0,True,False,False,False,12.766230,8.528529,0.000000,8.457159,12.779584,-345132.68,5.820766e-11,69.248305,1
733890,2224.85,2578.65,353.80,0.00,0.00,0,False,False,True,False,7.707894,7.855409,5.871554,0.000000,0.000000,0.00,-2.224850e+03,0.862462,0
656179,170020.34,2096782.65,2266802.99,2668162.89,2861708.82,0,False,False,False,False,12.043679,14.555915,14.633881,14.796901,14.866930,-340040.68,2.352559e+04,0.081086,0
347436,261231.80,2215631.50,2476863.31,493557.37,232325.57,0,False,False,False,False,12.473167,14.611048,14.722504,13.109396,12.355899,-522463.61,-5.224636e+05,0.117904,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154321,103368.58,103368.58,0.00,0.00,103368.58,1,True,False,False,False,11.546066,11.546066,0.000000,0.000000,11.546066,0.00,0.000000e+00,0.999990,1
570478,1735647.47,1735647.47,0.00,0.00,0.00,1,False,False,False,True,14.366892,14.366892,0.000000,0.000000,0.000000,0.00,-1.735647e+06,0.999999,1
615607,10000000.00,10000000.00,0.00,211048.11,10211048.11,1,True,False,False,False,16.118096,16.118096,0.000000,12.259846,16.138981,0.00,0.000000e+00,1.000000,1
220387,1171751.17,1171751.17,0.00,0.00,0.00,1,False,False,False,True,13.974011,13.974011,0.000000,0.000000,0.000000,0.00,-1.171751e+06,0.999999,1


In [98]:
#df_baseline.to_csv('../data/baseline_transactions.csv', index=False)

In [ ]:
# Save to CSV: not using SMOTE
# df_baseline
# also try to use df_baseline + get_dummies() encoding
# also df_baseline_log
# df_baseline_deltas
# also try df_baseline_log_hot_encoding
# also baseline_log_deltas_hot_encoding

# Save to CSV: using SMOTE
# df_baseline
# also try to use df_baseline + type encoding
# also df_baseline_log
# df_baseline_deltas
# also try df_baseline_log_hot_encoding
# also baseline_log_deltas_encoding

In [100]:
#df_log_transform.to_csv('../data/log_feature_transforms.csv', index=False)

In [106]:
#df_enhanced.to_csv('../data/log_diff_encode_features.csv', index=False)

In [ ]:
#df_baseline_hot_encode.to_csv('../data/baseline_hot_encode.csv, index=False.csv')